In [ ]:
import abdou_torch.abdou_torch
from abdou_torch.abdou_torch import Tensor, Linear, MLP, WAdam
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

In [ ]:

def main():
    X_data, y_data = make_blobs(n_samples=400, centers=2, n_features=2, cluster_std=1.4, random_state=42)

    y_data = 2 * y_data - 1
    y_data = y_data.reshape(-1, 1)

    X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
    )

    X_train = Tensor(X_train_data)
    y_train = Tensor(y_train_data)

    X_test = Tensor(X_test_data, requires_grad=False)
    y_test = Tensor(y_test_data, requires_grad=False)

    model = MLP(2, [16, 16, 1])

    opt = WAdam(model.parameters(), lr=0.05, weight_decay=0.01)

    print("Training...")

    steps = 1500
    N_train = len(X_train_data)
    for k in range(steps):

        scores = model(X_train)

        data_loss = ((scores - y_train)**2).sum() / N_train

        opt.zero_grad()
        data_loss.backward()

        opt.step()

        if k % 50 == 0:
            train_preds = np.where(scores.data > 0, 1, -1)
            train_acc = np.mean(train_preds == y_train.data)
            print(f"Step {k} | MSE Loss: {data_loss.data:.4f} | Train Acc: {train_acc*100:.1f}%")

    test_scores = model(X_test)
    test_preds = np.where(test_scores.data > 0, 1, -1)
    test_acc = np.mean(test_preds == y_test.data)
    print(f"\nFinal Test Accuracy (Unseen Data): {test_acc*100:.1f}%")

    print("Plotting results...")
    h = 0.1

    x_min, x_max = X_data[:, 0].min() - 1, X_data[:, 0].max() + 1
    y_min, y_max = X_data[:, 1].min() - 1, X_data[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))


    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    inputs = Tensor(mesh_points)

    Z = model(inputs)
    Z = Z.data.reshape(xx.shape)

    plt.figure(figsize=(8, 6))

    plt.contourf(xx, yy, Z, levels=np.linspace(-2, 2, 20), cmap='RdBu', alpha=0.6)
    plt.colorbar(label='Model Score')
    plt.contour(xx, yy, Z, levels=[0], colors='k', linestyles='--')

    plt.scatter(X_data[:, 0], X_data[:, 1], c=y_data.ravel(), cmap='RdBu', edgecolors='k', s=50)

    plt.title('abdou_torch - "Blobs" Classification (MSE)')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

In [ ]:
main()